# **Financial Machine Learning Assignment**

Student: B241413

Project choice: Forecasting Stock Volatility - Is stock volatility past-dependent?

Stock Choices:
1. Salesforce (CRM)
2. Goldman Sachs (GS)

In [1]:
#Import necessary libraries for analysis
import os
from os import listdir
import random
import tqdm
import numpy as np
import pandas as pd
import time

import tensorflow as tf
from tensorflow.python import keras
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras import backend as K

from keras.layers import Dense, Embedding, SimpleRNN
from keras.models import Sequential
from scipy import stats

from pprint import pprint
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
np.set_printoptions(suppress=True, precision=4)
os.environ['PYTHONHASHSEED'] = '0'

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import *
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from scipy import stats
import scipy.cluster.hierarchy as sch

%matplotlib inline

# These options determine the way floating point numbers are displayed.
np.set_printoptions(precision=4, suppress=True)

tf.random.set_seed(100)

2024-02-18 20:25:55.426298: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/var/folders/lg/tx14h9yd0y124jv8gq38m0mw0000gn/T/ipykernel_2978/3162438777.py:23: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


# Download stock from Yahoo Finance

In [2]:
### install and import yahoo finance
!pip install -U yfinance
import yfinance as yf

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/1b/0f/77716aa9dd84bb1aa5e93c87122af1de89697b6231f6d01d58d4e7c03c14/yfinance-0.2.36-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 4.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.5 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 3.5 MB/s eta 0:00:00
  Created wheel for frozendict: filename=frozendict-2.4.0-py3-none-any.whl size=15425 sha256=6fcc0bb55cd19f23e75dfe5c4ede4d7399975abad17a1a0e21ac0801692b2d96
  Stored in directory: /User

/Users/benblackwell/anaconda3/lib/python3.11/site-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [3]:
#### Define the stock Start and End Date ####
## you can define a longer time for your own use ##

start = pd.to_datetime('20000101',format='%Y%m%d')
end = pd.to_datetime('20231231',format='%Y%m%d')
days = (end-start).days
print('total days: ', days)

total days:  8765


In [19]:
#Reading the company listed stocks from the file 
# nasdaq_listed.csv contains all company tickers listed in nasdaq
df = pd.read_csv('/Users/benblackwell/Financial-Machine-Learning/nyse-listed.csv')
'''
find all company names listed on NASDAQ and download
the stoke prices from yahoo finance
'''
df.head()
Symbols = df['ACT Symbol'].tolist()

print('total number of tickers: ', len(Symbols))
print('first 10: ', Symbols[:10])



total number of tickers:  3298
first 10:  ['A', 'AA', 'AA$B', 'AAC', 'AAN', 'AAP', 'AAT', 'AAV', 'AB', 'ABB']


In [42]:
import pandas as pd
import yfinance as yf
import time

t0 = time.time()

# Define the stock tickers to retrieve prices for
tickers_to_download = ["GS", "CRM"]

# Define the time period for which you want to retrieve the prices
start = '2023-01-01'
end = '2024-01-01'

# Create empty dataframe
stock_final = pd.DataFrame()

# Empty list for the tickers that are not listed anymore
delisted_stock = []

# Iterate over each symbol
for ticker in tickers_to_download:
    print(f"Downloading data for {ticker}...", end=", ", flush=True)
    try:
        stock = yf.download(ticker, start=start, end=end, progress=False)
        if len(stock) == 0:
            delisted_stock.append(ticker)
        else:
            stock['Name'] = ticker
            stock_final = stock_final.append(stock, sort=False)
    except Exception:
        delisted_stock.append(ticker)

# Calculate total time taken
total_time = (time.time() - t0) / 60

print('\nTotal time:', total_time, 'minutes')
print('Total number of downloaded stocks:', len(stock_final['Name'].unique()))


Total time: 0.002595698833465576 minutes
Total number of downloaded stocks: 2


/var/folders/lg/tx14h9yd0y124jv8gq38m0mw0000gn/T/ipykernel_2978/372251870.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_final = stock_final.append(stock, sort=False)
/var/folders/lg/tx14h9yd0y124jv8gq38m0mw0000gn/T/ipykernel_2978/372251870.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_final = stock_final.append(stock, sort=False)


In [43]:
# Date index contains both date and time, we like to remove time information and use date information only

# make index as a normal column
stock_final.reset_index( inplace=True )

# remove time, leave dates only
stock_final['Date'] = pd.to_datetime(stock_final['Date'], format='%Y-%m-%d').dt.date

# set index as the date
stock_final.set_index('Date', inplace=True)

stock_final.head(10)

,Open,High,Low,Close,Adj Close,Volume,Name
Date,,,,,,,
2023-01-03,345.500000,348.600006,342.809998,346.220001,335.584076,1589700,GS
2023-01-04,348.029999,350.309998,345.079987,347.700012,337.018616,1881000,GS
2023-01-05,344.250000,345.140015,340.140015,343.760010,333.199646,1397800,GS
2023-01-06,347.299988,349.000000,340.950012,348.079987,337.386932,3097800,GS
2023-01-09,351.880005,356.500000,349.500000,353.000000,342.155792,1989000,GS
2023-01-10,352.640015,357.500000,350.500000,357.359985,346.381836,1610100,GS
2023-01-11,358.869995,365.500000,356.209991,364.480011,353.283142,2127000,GS
2023-01-12,366.239990,370.100006,362.570007,369.940002,358.575409,2103100,GS
2023-01-13,363.489990,374.350006,362.359985,374.000000,362.510651,3310200,GS


In [44]:
# !! if you have saved the data to your local place, no need to run this cell again !!

print('total stock downloaded: ', stock_final['Name'].nunique())

# save to Google Drive, you should change the directory to your own
stock_final.to_csv('/Users/benblackwell/Financial-Machine-Learning/nyse_downloaded_stocks.csv')

total stock downloaded:  2
